In [ ]:
import requests
import xmltodict
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET
import time 


def search(search_string):
    result = []
    start = 0 
    #This function passes your queries, separated by commas, in addition to the record you'd like to start with. 
    RECHERCHE_BASEURL = 'https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&startRecord=' + str(start)
    RECHERCHE_BASEURL1 = '&maximumRecords=100&page=' + str(start)
    RECHERCHE_BASEURL2 = '&query=%28gallica%20adj%20'
    
    search_string = search_string.replace('"','%22')
    search_string = search_string.replace(' ', '%20') # "%22gateaux%20de%20pommes%22"

    url = "".join([RECHERCHE_BASEURL, RECHERCHE_BASEURL1, RECHERCHE_BASEURL2, search_string]) #, START_REC])
   # print(url)
    s = requests.get(url, stream=True)
    soup = BeautifulSoup(s.content,"lxml-xml")
    tree = ET.ElementTree(ET.fromstring(s.text))
    root = tree.getroot()
    number_of_results=root.find('.//{http://www.loc.gov/zing/srw/}numberOfRecords')
   # print(number_of_results.text)
    number_of_pages = int(number_of_results.text) / 100
    
    for page in range(0,int(number_of_pages)): 
        page_no = str(page) 
    #    print(page_no)
        RECHERCHE_BASEURL = 'https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&startRecord=' + str(start) 
        RECHERCHE_BASEURL1 = '&maximumRecords=100&page=' + str(page)
        url = "".join([RECHERCHE_BASEURL, RECHERCHE_BASEURL1, RECHERCHE_BASEURL2, search_string]) #, START_REC])
    #    print(url)
        try:
            s = requests.get(url, stream=True)
            soup = BeautifulSoup(s.content,"lxml-xml")
            tree = ET.ElementTree(ET.fromstring(s.text))
            root = tree.getroot()
            article_id = ""
            articletype = ""
            title = ""
            publicationdate = "" 
            publicationname = ""
            page = "" 
            ocrquality = ""
            for child in root.findall('.//{http://www.openarchives.org/OAI/2.0/oai_dc/}dc'):
                for item in child:
                    if item.tag == '{http://purl.org/dc/elements/1.1/}title':
                        title = item.text
                    if item.tag == '{http://purl.org/dc/elements/1.1/}identifier':
                        if 'ark' in item.text:
                            article_id = item.text
                    if item.tag == '{http://purl.org/dc/elements/1.1/}date':
                        publicationdate = item.text
                    if item.tag == '{http://purl.org/dc/elements/1.1/}type':
                        articletype = item.text
                    if item.tag == '{http://www.kb.nl/ddd}papertitle':
                        publicationname = item.text
                    if item.tag == '{http://www.kb.nl/ddd}page':
                        page = item.text
                    if item.tag == 'ocrquality':
                        ocrquality = item.text 

                resultline = search_string + "\t" + article_id + "\t" + publicationdate + "\t" + title + "\t" + articletype + "\t" + publicationname + "\t" + page + "\n"
                result.append(resultline)
                s = {}
            sleep(1)
        except:
     #       print(s)
            pass
        start = start + 100
   # print("number_of_results ", len(result))
    return(result)






In [ ]:
def texte_brut(id):
    TEXTEBRUT_BASEURL = 'https://gallica.bnf.fr/ark:/'
    
    url = "".join([TEXTEBRUT_BASEURL, id, '.texteBrut'])
    #print(url)
    
    s = requests.get(url, stream=True)
    soup = BeautifulSoup(s.content,"lxml-xml")
    #print(soup.prettify())
    text= str(soup.hr)
    return(text)


def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
import os


In [ ]:
queries = ["tarte aux pommes","tarte de pommes", "tarte tatin", "gateau de pommes", "gateau aux pommes"]

os.chdir('/Users/marieke/Work/CommoditiesHandbook/french_documents')
print(os.getcwd())

for query in queries: 
    result = search(query)
    print(query, len(result))
   # outputfilename = '/Users/marieke/Work/CommoditiesHandbook/Query_results_' + query.replace(" ", "_") + '.tsv' 
   # f = open(outputfilename, 'w')
    for r in result:
   #     f.write(r)
        elems = r.split("\t")
        print(elems[1])
        parts = elems[1].split('/')
        if parts[5].startswith("bpt6") or parts[5].startswith("btv1"):
            id = parts[4] + "/" + parts[5]
   #     print(id)
            if not os.path.exists(parts[4]):
                os.makedirs(parts[4])
            os.chdir(parts[4])
            filename = elems[0].replace('%20', "_") + "_" + parts[5] + ".txt" 
            f = open(filename, 'w')
   #     print(filename)
            fulltext = texte_brut(id)
            clean_text = remove_html_tags(fulltext)
            f.write(elems[1] + "\n" + elems[2] + "\n")
            f.write(clean_text)
            f.close()
            os.chdir("/Users/marieke/Work/CommoditiesHandbook/french_documents")
       # print(os.getcwd())
            time.sleep(1)
        
